# Using BigQuery to perform basic data analytics

Here's a sample notebook of executing SQL commands in order to analyze some data, along with some basic visualization.

We'll explore some data sets and reproduce how we might write queries for certain business problems.

### Setup

In [3]:
# relevant installs
# !pip install google-cloud 
# !pip install --upgrade google-cloud-bigquery[pandas] 
# !pip install google-cloud-storage

In [4]:
%load_ext google.cloud.bigquery

In [5]:
SERVICE_ACCOUNT= 'bq_jupyter'
JSON_FILE_NAME = '../credentials/ds-portfolio-a04fdb631b73.json'
GCP_PROJECT_ID = 'ds-portfolio'

In [6]:
import subprocess
import sys
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

logger = logging.Logger('catch_all')

def run_command(parameters):
    try:
        # """Prints and runs a command."""
        return subprocess.check_output(parameters)
    except BaseException as e: 
       logger.error(e) 
       logger.error('ERROR: Looking in jupyter console for more information')

In [7]:
%matplotlib inline

### Queries

We'll be using the San Francisco Bikeshares dataset, which contains information around trips for the bikeshare program in San Francisco.

In [22]:
from google.cloud import bigquery

client = bigquery.Client.from_service_account_json(JSON_FILE_NAME)

def query_to_df(query):
    # transfers query results to pandas dataframe for easy manipulating
    return(client.query(QUERY).result().to_dataframe())

def get_schema(table):
    # retreives the schema as a printed object
    return(client.get_table(table).schema)

There's 4 different tables in this database. As a first step, we should look at the schema of all of these tables and see where we might be able to join for insight in future queries.

The tables are...
* bikeshare_regions
* bikeshare_station_info
* bikeshare_station_status
* bikeshare_trips

In [23]:
#bikeshare_regions
table = 'bigquery-public-data.san_francisco_bikeshare.bikeshare_regions'
get_schema(table)

[SchemaField('region_id', 'INTEGER', 'REQUIRED', 'Unique identifier for the region', ()),
 SchemaField('name', 'STRING', 'REQUIRED', 'Public name for this region', ())]

In [24]:
#bikeshare_station_info
table = 'bigquery-public-data.san_francisco_bikeshare.bikeshare_station_info'
get_schema(table)

[SchemaField('station_id', 'INTEGER', 'REQUIRED', 'Unique identifier of a station.', ()),
 SchemaField('name', 'STRING', 'REQUIRED', 'Public name of the station', ()),
 SchemaField('short_name', 'STRING', 'NULLABLE', 'Short name or other type of identifier, as used by the data publisher', ()),
 SchemaField('lat', 'FLOAT', 'REQUIRED', 'The latitude of station. The field value must be a valid WGS 84 latitude in decimal degrees format. See: http://en.wikipedia.org/wiki/World_Geodetic_System, https://en.wikipedia.org/wiki/Decimal_degrees', ()),
 SchemaField('lon', 'FLOAT', 'REQUIRED', 'The longitude of station. The field value must be a valid WGS 84 longitude in decimal degrees format. See: http://en.wikipedia.org/wiki/World_Geodetic_System, https://en.wikipedia.org/wiki/Decimal_degrees', ()),
 SchemaField('region_id', 'INTEGER', 'NULLABLE', 'ID of the region where station is located', ()),
 SchemaField('rental_methods', 'STRING', 'NULLABLE', 'Array of enumerables containing the payment me

In [26]:
#bikeshare_station_status
table = 'bigquery-public-data.san_francisco_bikeshare.bikeshare_station_status'
get_schema(table)

[SchemaField('station_id', 'INTEGER', 'REQUIRED', 'Unique identifier of a station', ()),
 SchemaField('num_bikes_available', 'INTEGER', 'REQUIRED', 'Number of bikes available for rental', ()),
 SchemaField('num_bikes_disabled', 'INTEGER', 'NULLABLE', 'Number of disabled bikes at the station. Vendors who do not want to publicize the number of disabled bikes or docks in their system can opt to omit station capacity (in station_information), num_bikes_disabled and num_docks_disabled. If station capacity is published then broken docks/bikes can be inferred (though not specifically whether the decreased capacity is a broken bike or dock)', ()),
 SchemaField('num_docks_available', 'INTEGER', 'REQUIRED', 'Number of docks accepting bike returns', ()),
 SchemaField('num_docks_disabled', 'INTEGER', 'NULLABLE', 'Number of empty but disabled dock points at the station. This value remains as part of the spec as it is possibly useful during development', ()),
 SchemaField('is_installed', 'BOOLEAN', 

In [27]:
#bikeshare_trips
table = 'bigquery-public-data.san_francisco_bikeshare.bikeshare_trips'
get_schema(table)

[SchemaField('trip_id', 'INTEGER', 'REQUIRED', 'Numeric ID of bike trip', ()),
 SchemaField('duration_sec', 'INTEGER', 'NULLABLE', 'Time of trip in seconds', ()),
 SchemaField('start_date', 'TIMESTAMP', 'NULLABLE', 'Start date of trip with date and time, in PST', ()),
 SchemaField('start_station_name', 'STRING', 'NULLABLE', 'Station name of start station', ()),
 SchemaField('start_station_id', 'INTEGER', 'NULLABLE', 'Numeric reference for start station', ()),
 SchemaField('end_date', 'TIMESTAMP', 'NULLABLE', 'End date of trip with date and time, in PST', ()),
 SchemaField('end_station_name', 'STRING', 'NULLABLE', 'Station name for end station', ()),
 SchemaField('end_station_id', 'INTEGER', 'NULLABLE', 'Numeric reference for end station', ()),
 SchemaField('bike_number', 'INTEGER', 'NULLABLE', 'ID of bike used', ()),
 SchemaField('zip_code', 'STRING', 'NULLABLE', 'Home zip code of subscriber (customers can choose to manually enter zip at kiosk however data is unreliable)', ()),
 Schema

So when we take a look at the schemas, we can see that each table gives us some different information. A few things that jump out:
* We get some interesting information from the `station_info` table regarding payment types. It could be interesting to look if stations with different payment types are associated with more or less rides.
* The `bikeshare_trips` table will give us information around ride-by-ride stats and has unique identifiers around customers/members that use them.
    * We have additional information for members, but not for customers
    * This will allow us to take a look at where popular routes might be
    
At this point, we can probably start looking at doing some queries for some explorative work, and seeing where we might be able to answer questions for real business impact.

In [ ]:
# q1

In [ ]:
QUERY = (
    """
    SELECT
      EXTRACT(YEAR
      FROM
        start_date) AS year,
      ROUND(AVG(duration_sec/60), 2) AS avg_duration_min,
      COUNT(*) AS yearly_trips
    FROM
      `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
    GROUP BY
      YEAR
    ORDER BY
      YEAR DESC
    """)

ret_df = client.query(QUERY).result().to_dataframe()
ret_df